In [1]:
year = 2023
day = 21

In [511]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
data = puzzle.examples[0].input_data

data = data.strip()
data = data.split("\n")
data = [list(d) for d in data]
data = np.array(data)
n = len(data)
print(n)

T = 1
start = (np.where(data == "S")[0][0], np.where(data == "S")[1][0])
data = np.tile(data, (T, T))

start = (start[0] + n * (T // 2), start[1] + n * (T // 2))
data[np.where(data == "S")] = "."

ratio = np.sum(np.sum(data == ".")) / np.prod(data.shape)

ratio


11


0.6694214876033058

In [512]:
import numpy as np

np.set_printoptions(edgeitems=30, linewidth=100000,
                    formatter=dict(float=lambda x: "%s" % x))

In [513]:
possible_spots = {start}

all_possible_spots = []

steps = []

H, W = data.shape

bbox = [0, 0, 0, 0]

for i in range(1, 501):
    new_possible_spots = set()
    for y, x in possible_spots:
        for dy, dx in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            y_, x_ = (y + dy), (x + dx)
            if data[y_ % H, x_ % W] == ".":
                new_possible_spots.add((y_, x_))
    possible_spots = new_possible_spots
    print(i, len(possible_spots))
    steps.append(len(possible_spots))
    all_possible_spots.append(list(possible_spots))
    # print(possible_spots)


1 2
2 4
3 6
4 9
5 13
6 16
7 22
8 30
9 41
10 50
11 63
12 74
13 89
14 99
15 115
16 129
17 145
18 165
19 192
20 216
21 234
22 261
23 294
24 326
25 353
26 395
27 427
28 460
29 491
30 537
31 574
32 605
33 644
34 689
35 740
36 784
37 846
38 894
39 944
40 989
41 1053
42 1107
43 1146
44 1196
45 1256
46 1324
47 1383
48 1464
49 1528
50 1594
51 1653
52 1735
53 1805
54 1853
55 1914
56 1988
57 2072
58 2145
59 2244
60 2324
61 2406
62 2479
63 2579
64 2665
65 2722
66 2794
67 2882
68 2982
69 3069
70 3186
71 3282
72 3380
73 3467
74 3585
75 3687
76 3753
77 3836
78 3938
79 4054
80 4155
81 4290
82 4402
83 4516
84 4617
85 4753
86 4871
87 4946
88 5040
89 5156
90 5288
91 5403
92 5556
93 5684
94 5814
95 5929
96 6083
97 6217
98 6301
99 6406
100 6536
101 6684
102 6813
103 6984
104 7128
105 7274
106 7403
107 7575
108 7725
109 7818
110 7934
111 8078
112 8242
113 8385
114 8574
115 8734
116 8896
117 9039
118 9229
119 9395
120 9497
121 9624
122 9782
123 9962
124 10119
125 10326
126 10502
127 10680
128 10837
129 11045

In [514]:
lens = np.array([len(p) for p in all_possible_spots])
ref = (np.array(np.arange(len(lens))**2)*ratio).astype("int")
print(lens - ref)

[  2   4   4   3   3   0  -2  -2  -1  -4  -3  -7  -7 -14 -16 -21 -26 -28 -24 -25 -33 -34 -30 -28 -32 -23 -25 -28 -33 -25 -28 -38 -41 -40 -33 -36 -21 -22 -22 -29 -18 -18 -34 -41 -40 -31 -33 -14 -14 -13 -20  -6  -5 -27 -38 -37 -27 -29  -7  -6  -3 -11   6   9 -19 -34 -34 -23 -26  -1   2   6  -3  18  22 -12 -30 -31 -18 -22   6  10  15   6  30  35  -5 -26 -28 -14 -19  13  19  25  14  42  48   3 -23 -25 -10 -15  20  27  34  23  54  61  10 -19 -22  -5 -12  27  35  43  32  66  74  18 -15 -19  -1  -8  33  43  53  40  78  88  25 -11 -16   3  -5  40  51  62  49  90 101  33  -8 -13   7  -1  47  59  71  58 102 114  40  -4 -10  12   2  54  67  81  66 113 127  48   0  -7  16   6  61  75  90  75 125 140  55   3  -4  20  10  68  83 100  84 137 154  63   7  -1  24  13  74  91 109  92 149 167  70  11   2  29  17  81  99 118 101 161 180  77  15   5  33  20  88 108 128 109 173 193  85  18   8  37  24  95 116 137 118 185 206  92  22  11  42  27 102 124 146 127 197 219 100  26  14  46  31 108 132 156 135 209

In [515]:
box_filter = lambda by, bx: lambda loc: (loc[0] >= 0+by*H) and (loc[0] < H+by*H) and (loc[1] >= 0+bx*W) and (loc[1] < W+bx*W)

box_repeats = {}

Z = 5
for by in range(-Z, Z+1):
    for bx in range(-Z, Z+1):
        lens = np.array([len(list(filter(box_filter(by, bx), l))) for l in all_possible_spots])
        # print(by, bx, lens)
        repeat_index = np.where(lens == 42)[0][0]
        print(by, bx, repeat_index)
        box_repeats[(by, bx)] = repeat_index
        # check that it actually repeats as expected
        # right = lens[repeat_index:]
        # print(np.all(right == np.tile(np.array([42,39]), len(right)//2+1)[:len(right)]))

42

In [516]:
start_offset = 35
outputs = []
for i in range(1, 6):
    outputs.append(list(sorted(map(lambda l: (l[0]%H, l[1]%W), filter(box_filter(i+1, i+1), all_possible_spots[start_offset+i*22])))))

-5 -5 119
-5 -4 108
-5 -3 97
-5 -2 86
-5 -1 75
-5 0 68
-5 1 75
-5 2 86
-5 3 97
-5 4 108
-5 5 119
-4 -5 108
-4 -4 97
-4 -3 86
-4 -2 75
-4 -1 64
-4 0 57
-4 1 64
-4 2 75
-4 3 86
-4 4 97
-4 5 108
-3 -5 97
-3 -4 86
-3 -3 75
-3 -2 64
-3 -1 53
-3 0 46
-3 1 53
-3 2 64
-3 3 75
-3 4 86
-3 5 97
-2 -5 86
-2 -4 75
-2 -3 64
-2 -2 53
-2 -1 42
-2 0 35
-2 1 42
-2 2 53
-2 3 64
-2 4 75
-2 5 86
-1 -5 75
-1 -4 64
-1 -3 53
-1 -2 42
-1 -1 31
-1 0 24
-1 1 31
-1 2 42
-1 3 53
-1 4 64
-1 5 75
0 -5 70
0 -4 59
0 -3 48
0 -2 37
0 -1 22
0 0 13
0 1 26
0 2 39
0 3 50
0 4 61
0 5 72
1 -5 75
1 -4 64
1 -3 53
1 -2 42
1 -1 31
1 0 28
1 1 35
1 2 46
1 3 57
1 4 68
1 5 79
2 -5 86
2 -4 75
2 -3 64
2 -2 53
2 -1 42
2 0 39
2 1 46
2 2 57
2 3 68
2 4 79
2 5 90
3 -5 97
3 -4 86
3 -3 75
3 -2 64
3 -1 53
3 0 50
3 1 57
3 2 68
3 3 79
3 4 90
3 5 101
4 -5 108
4 -4 97
4 -3 86
4 -2 75
4 -1 64
4 0 61
4 1 68
4 2 79
4 3 90
4 4 101
4 5 112
5 -5 119
5 -4 108
5 -3 97
5 -2 86
5 -1 75
5 0 72
5 1 79
5 2 90
5 3 101
5 4 112
5 5 123


In [517]:
assert outputs[0] == outputs[1]
assert outputs[1] == outputs[2]
assert outputs[2] == outputs[3]
assert outputs[3] == outputs[4]


In [518]:
idx = 33
list(sorted(map(lambda l: (l[0]%H, l[1]%W), filter(box_filter(0, 2), all_possible_spots[idx]))))

In [519]:
from collections import defaultdict
box_spots = defaultdict(list)
for l in all_possible_spots:
    curr_box_spots = {}

    print(len(l))

2

In [523]:
for by in range(-Z, Z+1):
    for bx in range(-Z, Z+1):
        ref_by = (by/abs(by) + max(0, abs(by)-1))*(by/abs(by))
        ref_bx = (bx/abs(bx) + max(0, abs(bx)-1))*(bx/abs(bx))
        assert box_repeats[(by, bx)] == box_repeats[, bx/abs(bx))] +
        box_repeats[(by, bx)]

2409214
2409212
2409213
2409214
2409215
2409216
2409217
2409218
2409219
2409220
2409221
2409222
2409223


470149643712804

In [488]:
np.tile(np.array([42,39]), 3)

4

In [494]:
np.where(np.array(lens) == 39)

3
3
3
3
4
4
4
4
4
5
5
5
5
5
5
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7


1060

In [409]:
# box_filter = lambda by, bx: lambda loc: (loc[0] >= 0+by*H) and (loc[0] < H+by*H) and (loc[1] >= 0+bx*W) and (loc[1] < W+bx*W)
#
#
# for by in range(-1, 2):
#     for bx in range(-1, 2):
#         prev = None
#         prevprev = None
#         N = None
#         for i, l in enumerate(all_possible_spots):
#             prevprev = prev
#             prev = N
#             N = len(list(filter(box_filter(by, bx), l)))
#             if N == prevprev and N > 35:
#                 print(by, bx, i)
#                 break


set()

In [410]:
# possible_spots = np.zeros(data.shape, dtype=int)
# possible_spots[*start] = 1
#
# steps = []
#
# H, W = data.shape
#
# for i in range(1, 11):
#     new_possible_spots = np.zeros(data.shape, dtype=int)
#     for y, x in all_coords:
#         curr = possible_spots[y, x]
#         for dy, dx in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
#             y_, x_ = (y + dy) % H, (x + dx) % W
#             if data[y_, x_] == ".":
#                 new_possible_spots[y_, x_] += curr
#     possible_spots = new_possible_spots
#     print(i, int(np.sum(np.sum(possible_spots))))
#     steps.append(len(possible_spots))
#     print(possible_spots)


set()

167006

In [ ]:
1, 4, 9, 16, 25, 36, 49, 64, 81, 100

In [210]:
np.sum(np.sum(data == "."))

15550

In [211]:
np.sum(np.sum(data == "#"))

1611

In [212]:
26501365**2 * (15550 / (15550+1611))

636391381255355.0

In [ ]:
# 636391381255355